<a href="https://colab.research.google.com/github/chicago007/test2/blob/main/%EB%8B%A8%EA%B8%B0%EC%B6%94%EC%84%B8%EC%A2%85%EB%AA%A9_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 18.1 MB/s eta 0:00:00


In [ ]:
# prompt: pykrx에서 아래조건에 해당되는 종목을 가져올 수 있게 작성해줘
# A : 비율내림차순({거래대금}) >= 95
# B : {일별주가상승률} < 28
# C : 과거값('변화율_기간({종가},{10일})',{1일}) < 35
# D : {KOSDAQ_소형지수_종가}/이동평균({KOSDAQ_소형지수_종가},{60일}) < 1
# E : 비율내림차순({시가총액(원)}) < 90
# F : {거래량}/이동평균({거래량},{5일}) < 5
# A & E & C & F and (D or B)

In [53]:
# prompt: pykrx에서 아래조건에 해당되는 종목을 가져올 수 있게 작성해줘
# 비율내림차순({거래대금}) >= 95

from pykrx import stock
import pandas as pd
from datetime import datetime, timedelta

def find_stocks(date):

    # pykrx market_cap에는 아래 데이터가 포함되어 있음
    # 티커 / 종가, 시가총액, 거래량, 거래대금, 상장주식수
    # KOSPI와 KOSDAQ 상장종목만 가져온다
    df1 = stock.get_market_cap(date, market='KOSPI')
    df2 = stock.get_market_cap(date, market='KOSDAQ')
    df = pd.concat([df1, df2])
    # 우선주를 제외해 준다
    df = df[df.index.str.contains('0$', regex=True)]

    # A. 시가총액 기준으로 내림차순 정렬 후 상위 10%이하 종목 선택
    df_market_cap = df.sort_values(by='시가총액', ascending=False)
    # 시총 90% 이하
    df_market_cap90 = df_market_cap.iloc[int(len(df) * 0.1):]
    market_cap_tickers =df_market_cap90.index.tolist()

    # B. 거래대금 기준으로 내림차순 정렬 후 상위 5% 종목 선택
    df_td_volume = df.sort_values(by='거래대금', ascending=False)
    df_td_volume5 = df_td_volume.iloc[:int(len(df_td_volume) * 0.05)]
    td_volume_tickers = df_td_volume5.index.tolist()

    # A&B. 시가총액 상위 10%와 거래대금 상위 5% 교차 종목 찾기
    filter1_tickers = list(set(market_cap_tickers) & set(td_volume_tickers))
    print(len(filter1_tickers))
    # print(filter1_tickers)
    # print(type(final_tickers[0]))

    # C. A&B에서 필터링된 종목들의 5일평균 거래량과 전일기준 10일 수익률 계산
    # 전일기준 10일 수익률 35% 미만, 1일거래량이 5일평균거래량의 5배 미만인 종목
    end_date = date
    start_date = (datetime.today() - timedelta(days=30)).strftime("%Y%m%d")
    result = []
    for ticker in filter1_tickers:
        df_ohlcv = stock.get_market_ohlcv(start_date, end_date, ticker)
        df_ohlcv['5일평균거래량'] = df_ohlcv['거래량'].rolling(window=5).mean()
        df_ohlcv['1일수익률'] = df_ohlcv['종가'].pct_change(periods=1) * 100
        df_ohlcv['10일수익률'] = df_ohlcv['종가'].pct_change(periods=10) * 100
        # print(ticker)
        # print(df_ohlcv)

        # Check if df_ohlcv has enough rows and meets the conditions
        if len(df_ohlcv) >= 12 and df_ohlcv['10일수익률'].iloc[-2] < 35 and df_ohlcv['거래량'].iloc[-1] < df_ohlcv['5일평균거래량'].iloc[-1] * 5:
            result.append(
                {
                    '티커': ticker,
                    '종목명': stock.get_market_ticker_name(ticker),
                    '1일거래량': df_ohlcv['거래량'].iloc[-1],
                    '5일평균거래량': df_ohlcv['5일평균거래량'].iloc[-1],
                    '1일수익률': df_ohlcv['1일수익률'].iloc[-1],
                    '10일수익률': df_ohlcv['10일수익률'].iloc[-2]
                },
            )
    df_return = pd.DataFrame(result)
    # print(df_return)

    # D. 코스닥 소형지수 60일평균보다 작으면 그냥 기존 종목에서 선정하고
    # 아니면 1일 수익률이 28% 이하인 종목만 선정

    end_date = date
    start_date = (datetime.today() - timedelta(days=90)).strftime("%Y%m%d")
    df = stock.get_index_ohlcv(start_date, end_date, "2004")
    df['60일평균'] = df['종가'].rolling(window=60).mean()
    if df['종가'].iloc[-1] < df['60일평균'].iloc[-1]:
        final_tickers = df_return['티커'].tolist()
        print(1)
        # print(final_tickers)
    else:
        final_tickers = df_return[df_return['1일수익률'] < 28]['티커'].tolist()
        print(2)
        # print(final_tickers)


    df_final = df_return[df_return['티커'].isin(final_tickers)]
    # print(df_final)

    final_df = df_final.sort_values(by=['1일수익률'], ascending=False)
    # print(final_df)


    return final_df

# 사용 예시 (날짜를 원하는 날짜로 변경)
date = "20250306"
top_stocks = find_stocks(date)
# print(top_stocks.head(50))

# top_stocks DataFrame을 'top_stocks.xlsx' 파일로 저장합니다.
# top_stocks.to_excel('top_stocks.xlsx', index=False)

# 종목코드와 종목명만 선택하여 새로운 DataFrame을 생성합니다.
selected_columns = top_stocks[['티커', '종목명']]
print(selected_columns)

new_date = "20250307"
result = []
for ticker in top_stocks['티커']:
    df_ohlcv = stock.get_market_ohlcv(date, new_date, ticker)
    df_ohlcv['시/종'] = (df_ohlcv['시가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
    df_ohlcv['고/종'] = (df_ohlcv['고가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
    df_ohlcv['저/종'] = (df_ohlcv['저가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
    df_ohlcv['종/종'] = (df_ohlcv['종가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
    df_ohlcv['거래량증가율'] = (df_ohlcv['거래량'].iloc[-1] / df_ohlcv['거래량'].iloc[-2] - 1) * 100
    df_ohlcv['고/시'] = (df_ohlcv['고가'].iloc[-1] / df_ohlcv['시가'].iloc[-1] - 1) * 100
    df_ohlcv['저/시'] = (df_ohlcv['저가'].iloc[-1] / df_ohlcv['시가'].iloc[-1] - 1) * 100
    df_ohlcv['종/시'] = (df_ohlcv['종가'].iloc[-1] / df_ohlcv['시가'].iloc[-1] - 1) * 100

    result.append(
        {
            '티커': ticker,
            '종목명': stock.get_market_ticker_name(ticker),
            '시/종': df_ohlcv['시/종'].iloc[-1],
            '고/종': df_ohlcv['고/종'].iloc[-1],
            '저/종': df_ohlcv['저/종'].iloc[-1],
            '종/종': df_ohlcv['종/종'].iloc[-1],
            '고/시': df_ohlcv['고/시'].iloc[-1],
            '저/시': df_ohlcv['저/시'].iloc[-1],
            '종/시': df_ohlcv['종/시'].iloc[-1],
            '전일거개량' : df_ohlcv['거래량'].iloc[-2],
            '당일거래량' : df_ohlcv['거래량'].iloc[-1],
            '거래량증가율': df_ohlcv['거래량증가율'].iloc[-1],
        },
    )
df_final = pd.DataFrame(result)
print(df_final)
df_final.to_excel('final.xlsx', index=False)

37
2
        티커        종목명
3   112290       와이씨켐
14  456010     아이씨티케이
22  010820        퍼스텍
1   065500     오리엔트정공
11  306200       세아제강
23  241520  DSC인베스트먼트
6   189300     인텔리안테크
12  452450       피아이이
20  452300    캡스톤파트너스
27  214420       토니모리
13  009520      포스코엠텍
28  039610       화성밸브
25  199550      레이저옵텍
2   439090       마녀공장
17  024740       한일단조
0   452190      한빛레이저
19  161580       필옵틱스
24  037270    YG PLUS
4   079900     전진건설로봇
21  097230      HJ중공업
7   437730         삼현
9   466100        클로봇
8   474610     RF시스템즈
10  008830       대동기어
15  358570   지아이이노베이션
5   030530      원익홀딩스
16  051980     중앙첨단소재
26  001570         금양


<ipython-input-53-739f6568a281>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ohlcv['시/종'] = (df_ohlcv['시가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
<ipython-input-53-739f6568a281>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ohlcv['고/종'] = (df_ohlcv['고가'].iloc[-1] / df_ohlcv['종가'].iloc[-2] - 1) * 100
<ipython-input-53-739f6568a281>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



        티커        종목명       시/종        고/종        저/종        종/종        고/시  \
0   112290       와이씨켐 -5.108055  -2.554028 -13.359528 -12.377210   2.691511   
1   456010     아이씨티케이 -0.997506   2.078138  -2.992519  -2.576891   3.106633   
2   010820        퍼스텍 -0.369458  -0.123153  -7.142857  -4.926108   0.247219   
3   065500     오리엔트정공 -1.818182   8.311688 -18.181818  -2.727273  10.317460   
4   306200       세아제강  0.000000   2.525253  -5.353535  -1.717172   2.525253   
5   241520  DSC인베스트먼트  0.194932   4.093567  -6.042885  -3.703704   3.891051   
6   189300     인텔리안테크 -2.588235   0.588235  -5.647059  -3.647059   3.260870   
7   452450       피아이이 -4.295704  11.288711  -5.394605  -4.295704  16.283925   
8   452300    캡스톤파트너스 -1.554404   4.663212  -5.008636  -3.626943   6.315789   
9   214420       토니모리  0.000000   6.538895  -4.284104  -2.593010   6.538895   
10  009520      포스코엠텍 -2.564103  16.776557  -2.637363   7.765568  19.849624   
11  039610       화성밸브 -0.786026   7.074236  -2.62008

In [44]:
from pykrx import stock
import pandas as pd

# pykrx market_cap에는 아래 데이터가 포함되어 있음
# 티커 / 종가, 시가총액, 거래량, 거래대금, 상장주식수

# KOSPI와 KOSDAQ 상장종목만 가져온다
df1 = stock.get_market_cap('20250307', market='KOSPI')
df2 = stock.get_market_cap('20250307', market='KOSDAQ')
df = pd.concat([df1, df2])
# 우선주를 제외해 준다
# df = df[df.index.str.contains('0$', r

for ticker in stock.get_index_ticker_list(market="KOSDAQ"):
    print(ticker, stock.get_index_ticker_name(ticker))

2001 코스닥
2002 코스닥 대형주
2003 코스닥 중형주
2004 코스닥 소형주
2012 일반서비스
2024 제조
2026 건설
2027 유통
2029 운송·창고
2031 금융
2037 오락·문화
2056 음식료·담배
2058 섬유·의류
2062 종이·목재
2063 출판·매체복제
2065 화학
2066 제약
2067 비금속
2068 금속
2070 기계·장비
2072 전기전자
2074 의료·정밀기기
2075 운송장비·부품
2077 기타제조
2114 통신
2118 IT 서비스
2181 코스닥 우량기업부
2182 코스닥 벤처기업부
2183 코스닥 중견기업부
2184 코스닥 기술성장기업부
2189 코스닥 글로벌
2203 코스닥 150
2212 코스닥 150 소재
2213 코스닥 150 산업재
2214 코스닥 150 필수소비재
2215 코스닥 150 자유소비재
2216 코스닥 150 정보기술
2217 코스닥 150 헬스케어
2218 코스닥 150 커뮤니케이션서비스
